# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
import pprint
import numpy as np
import pickle

import re
import nltk
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet') # download for lemmatization
# nltk.download('averaged_perceptron_tagger')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import pos_tag, ne_chunk
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import confusion_matrix


In [2]:
# load data from database
engine = create_engine('sqlite:///data/DisasterResponse.db')
table_name = engine.table_names()[0]
df = pd.read_sql_table(table_name, con=engine)
X = df['message'].values
labels = list(df.columns[5:])
y = df[labels].values

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    # convert to lowercase
    text = text.lower() 
    # remove punctuation characters
    text = re.sub(r"[^a-zA-Z0-9]", " ", text) 
    # words tokenization
    tokens = word_tokenize(text)
    # remove stopwords
    tokens_stopwords_removed = [w for w in tokens if w not in stopwords.words("english")]
    # reduce words to their root form performing Lemmatisation
    clean_tokens = [WordNetLemmatizer().lemmatize(w) for w in tokens_stopwords_removed]
    # reduce words to their word stem performing Stemming
    # clean_tokens = [PorterStemmer().stem(w) for w in tokens_stopwords_removed]
    return clean_tokens

In [4]:
# test
for message in X[:3]:
    tokens = tokenize(message)
    print(message)
    print(tokens, '\n')

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti'] 

Is the Hurricane over or is it not over
['hurricane'] 

Looking for someone but no name
['looking', 'someone', 'name'] 



### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('moclf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [7]:
# Train classifier
pipeline.fit(X_train, y_train)

KeyboardInterrupt: 

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
y_pred = pipeline.predict(X_test)

In [9]:
def display_results(y_test, y_pred):
    accuracy = (y_pred == y_test).mean()
    print("Accuracy:", accuracy, "\n")
    for i, column_name in enumerate(labels):
        print(column_name, classification_report(y_test[:, i], y_pred[:, i]))

In [10]:
display_results(y_test, y_pred)

Accuracy: 0.9493266397468079 

request               precision    recall  f1-score   support

           0       0.89      0.98      0.93      5394
           1       0.81      0.42      0.56      1151

   micro avg       0.88      0.88      0.88      6545
   macro avg       0.85      0.70      0.74      6545
weighted avg       0.87      0.88      0.87      6545

offer               precision    recall  f1-score   support

           0       1.00      1.00      1.00      6518
           1       0.00      0.00      0.00        27

   micro avg       1.00      1.00      1.00      6545
   macro avg       0.50      0.50      0.50      6545
weighted avg       0.99      1.00      0.99      6545

aid_related               precision    recall  f1-score   support

           0       0.75      0.85      0.80      3814
           1       0.75      0.61      0.67      2731

   micro avg       0.75      0.75      0.75      6545
   macro avg       0.75      0.73      0.74      6545
weighted avg     

C:\Users\gobro\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.94      1.00      0.97      6143
           1       0.37      0.02      0.03       402

   micro avg       0.94      0.94      0.94      6545
   macro avg       0.65      0.51      0.50      6545
weighted avg       0.90      0.94      0.91      6545

transport               precision    recall  f1-score   support

           0       0.96      1.00      0.98      6255
           1       0.63      0.10      0.17       290

   micro avg       0.96      0.96      0.96      6545
   macro avg       0.80      0.55      0.58      6545
weighted avg       0.95      0.96      0.94      6545

buildings               precision    recall  f1-score   support

           0       0.96      1.00      0.98      6211
           1       0.69      0.15      0.25       334

   micro avg       0.95      0.95      0.95      6545
   macro avg       0.83      0.57      0.61      6545
weighted avg       0.94      0.95      0.94      6545


### 6. Improve your model
Use grid search to find better parameters. 

In [27]:
pp = pprint.PrettyPrinter(indent=1)
pp.pprint(pipeline.get_params())

{'memory': None,
 'moclf': MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None),
 'moclf__estimator': RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
 'moclf__estimator__bootstrap': True,
 'moclf

In [9]:
parameters = {
        'moclf__estimator__n_estimators': [10, 100]
    }

cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=1, cv=3, verbose=True)

In [10]:
cv.fit(X_train, y_train)

Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed: 39.0min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip..._score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=1,
       param_grid={'moclf__estimator__n_estimators': [10, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=True)

In [162]:
cv.grid_scores_

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
y_pred = pipeline.predict(X_test)

In [23]:
# print("\nBest Parameters:", cv.best_params_)


Best Parameters: {'moclf__estimator__bootstrap': True}


In [21]:
# accuracy = (y_pred == y_test).mean()
# print("Accuracy:", accuracy)

Accuracy: 0.945186145865


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            try:
                first_word, first_tag = pos_tags[0]
                if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                    return True
            except:
                return False
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [216]:
class TextLengthExtractor(BaseEstimator, TransformerMixin):

    def get_length(self, text):
        words = nltk.word_tokenize(text)
        words = [w for w in words if w not in stopwords.words("english")]
        return len(words)

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.get_length)
        return pd.DataFrame(X_tagged)

In [207]:
# Test
StartingVerb  = StartingVerbExtractor()
TextLength = TextLengthExtractor()
TextLength.transform(X_train[0:10])

In [210]:
pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),
        
            ('text_length', Pipeline([
                ('count', TextLengthExtractor())
            ]))
            
        ])),

        ('moclf', MultiOutputClassifier(RandomForestClassifier()))
    ])

In [211]:
# Train classifier
pipeline.fit(X_train, y_train)

C:\Users\gobro\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\gobro\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\gobro\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\gobro\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\gobro\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: F

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=None,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, ma..._score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None))])

In [212]:
y_pred = pipeline.predict(X_test)

In [226]:
display_results(y_test, y_pred)

Accuracy: 0.9492611590090582 

request               precision    recall  f1-score   support

           0       0.89      0.98      0.93      5437
           1       0.82      0.42      0.55      1108

   micro avg       0.89      0.89      0.89      6545
   macro avg       0.86      0.70      0.74      6545
weighted avg       0.88      0.89      0.87      6545

offer               precision    recall  f1-score   support

           0       1.00      1.00      1.00      6514
           1       0.00      0.00      0.00        31

   micro avg       1.00      1.00      1.00      6545
   macro avg       0.50      0.50      0.50      6545
weighted avg       0.99      1.00      0.99      6545

aid_related               precision    recall  f1-score   support

           0       0.76      0.86      0.80      3893
           1       0.74      0.59      0.66      2652

   micro avg       0.75      0.75      0.75      6545
   macro avg       0.75      0.72      0.73      6545
weighted avg     

weather_related               precision    recall  f1-score   support

           0       0.86      0.95      0.91      4752
           1       0.82      0.60      0.70      1793

   micro avg       0.86      0.86      0.86      6545
   macro avg       0.84      0.78      0.80      6545
weighted avg       0.85      0.86      0.85      6545

floods               precision    recall  f1-score   support

           0       0.95      1.00      0.97      6034
           1       0.91      0.38      0.53       511

   micro avg       0.95      0.95      0.95      6545
   macro avg       0.93      0.69      0.75      6545
weighted avg       0.95      0.95      0.94      6545

storm               precision    recall  f1-score   support

           0       0.94      0.98      0.96      5933
           1       0.73      0.40      0.52       612

   micro avg       0.93      0.93      0.93      6545
   macro avg       0.84      0.69      0.74      6545
weighted avg       0.92      0.93      0.92  

### 9. Export your model as a pickle file


In [ ]:
# save the model to disk
filename = 'model.sav'
pickle.dump(model, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.